# Feature engineering experiments

Goal: test simple survey-structure features (no combinations with other methods yet).

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

ROOT = Path("..").resolve()
DATA_DIR = ROOT / "data"

X = pd.read_csv(DATA_DIR / "training_set_features.csv", index_col="respondent_id")
y = pd.read_csv(DATA_DIR / "training_set_labels.csv", index_col="respondent_id")

strat = (2 * y["h1n1_vaccine"].astype(int) + y["seasonal_vaccine"].astype(int)).values


def build_features(df: pd.DataFrame, add_engineered: bool) -> pd.DataFrame:
    Xf = df.copy()
    if not add_engineered:
        return Xf

    # Behavioral sum (binary behaviors)
    behavioral_cols = [
        "behavioral_antiviral_meds",
        "behavioral_avoidance",
        "behavioral_face_mask",
        "behavioral_wash_hands",
        "behavioral_large_gatherings",
        "behavioral_outside_home",
        "behavioral_touch_face",
    ]
    Xf["behavioral_sum"] = Xf[behavioral_cols].sum(axis=1, skipna=True)

    # Opinion gaps between H1N1 and seasonal
    Xf["opinion_effective_gap"] = Xf["opinion_h1n1_vacc_effective"] - Xf["opinion_seas_vacc_effective"]
    Xf["opinion_risk_gap"] = Xf["opinion_h1n1_risk"] - Xf["opinion_seas_risk"]
    Xf["opinion_sick_gap"] = Xf["opinion_h1n1_sick_from_vacc"] - Xf["opinion_seas_sick_from_vacc"]

    # Combined doctor recommendation indicator
    Xf["doctor_recc_any"] = (
        (Xf["doctor_recc_h1n1"] == 1) | (Xf["doctor_recc_seasonal"] == 1)
    ).astype(float)

    return Xf


def eval_lgbm(X_raw: pd.DataFrame, y: pd.DataFrame, seed: int = 42) -> dict:
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    oof = pd.DataFrame(index=y.index, columns=y.columns, dtype=float)
    for target in y.columns:
        oof[target] = 0.0

    cat_cols = X_raw.select_dtypes(include=["object"]).columns.tolist()
    cat_indices = [X_raw.columns.get_loc(c) for c in cat_cols]

    for tr_idx, va_idx in skf.split(X_raw, strat):
        X_tr, X_va = X_raw.iloc[tr_idx].copy(), X_raw.iloc[va_idx].copy()
        for c in cat_cols:
            X_tr[c] = X_tr[c].astype("category")
            X_va[c] = X_va[c].astype("category")

        for target in y.columns:
            model = lgb.LGBMClassifier(
                n_estimators=800,
                learning_rate=0.05,
                num_leaves=64,
                max_depth=-1,
                min_data_in_leaf=50,
                feature_fraction=0.8,
                bagging_fraction=0.8,
                bagging_freq=1,
                objective="binary",
                n_jobs=4,
            )
            model.fit(
                X_tr,
                y[target].iloc[tr_idx],
                categorical_feature=cat_indices,
            )
            oof.loc[X_va.index, target] = model.predict_proba(X_va)[:, 1]

    scores = {col: roc_auc_score(y[col], oof[col]) for col in y.columns}
    scores["mean_auc"] = float(np.mean(list(scores.values())))
    return scores

base_scores = eval_lgbm(build_features(X, add_engineered=False), y)
engineered_scores = eval_lgbm(build_features(X, add_engineered=True), y)

base_scores, engineered_scores

C:\Users\Admin\AppData\Local\Temp\ipykernel_4820\2209482061.py:54: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = X_raw.select_dtypes(include=["object"]).columns.tolist()


[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Number of positive: 4539, number of nega

C:\Users\Admin\AppData\Local\Temp\ipykernel_4820\2209482061.py:54: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = X_raw.select_dtypes(include=["object"]).columns.tolist()


[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Number of positive: 4539, number of nega

({'h1n1_vaccine': 0.8571762140702375,
  'seasonal_vaccine': 0.8527685387783289,
  'mean_auc': 0.8549723764242831},
 {'h1n1_vaccine': 0.8580834947234753,
  'seasonal_vaccine': 0.8534637176095968,
  'mean_auc': 0.8557736061665361})